In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import minmax_scale
from scipy.interpolate import CubicSpline

In [2]:
path = 'C:\\Users\\joanb\\Desktop\\data_incendios\\clima\\raw\\CLEAN_DATA'
os.listdir(path)

['data_Alzira',
 'data_Artana',
 'data_Beneixama',
 'data_Beniarda',
 'data_Bolulla',
 'data_Carcaixent',
 'data_Chella',
 'data_Gatova',
 'data_Llutxent',
 'data_Montan',
 'data_ValldEbo',
 'data_VallGallinera']

In [3]:
df_time = pd.read_csv('time_init_end.csv')

In [4]:
df_time['Inicio'] = pd.to_datetime(df_time['Dia Inicio'] + ' ' + df_time['Hora inicio'], format='%Y/%m/%d %H:%M')
df_time['Fin'] = pd.to_datetime(df_time['Dia Fin'] + ' ' + df_time['Hora fin'], format='%Y/%m/%d %H:%M')

In [5]:
df_time = df_time[['Incendio', 'Inicio', 'Fin']]
df_time.head()

,Incendio,Inicio,Fin
0,Artana,2016-07-25 15:00:00,2016-07-27 12:00:00
1,Beneixama,2019-07-15 14:00:00,2019-07-15 22:00:00
2,Beniarda,2020-08-28 21:00:00,2020-08-29 18:30:00
3,Bolulla,2016-09-05 14:00:00,2016-09-06 19:00:00
4,Carcaixent,2016-06-16 17:00:00,2016-06-18 09:00:00


Añadimos para el incendio de Alzira que es el de Carcaixent!

In [6]:
df_time_aux = pd.DataFrame(index=range(df_time.shape[0]+1), columns=df_time.columns)
df_time_aux.iloc[0] = df_time.iloc[4]
df_time_aux.iloc[1:] = df_time.iloc[:]
df_time = df_time_aux.copy()
# hay que reordenar las filas por el orden de los archivos.
temp_row = df_time.iloc[10].copy()  
df_time.iloc[10] = df_time.iloc[11]  
df_time.iloc[11] = temp_row
df_time

,Incendio,Inicio,Fin
0,Carcaixent,2016-06-16 17:00:00,2016-06-18 09:00:00
1,Artana,2016-07-25 15:00:00,2016-07-27 12:00:00
2,Beneixama,2019-07-15 14:00:00,2019-07-15 22:00:00
3,Beniarda,2020-08-28 21:00:00,2020-08-29 18:30:00
4,Bolulla,2016-09-05 14:00:00,2016-09-06 19:00:00
5,Carcaixent,2016-06-16 17:00:00,2016-06-18 09:00:00
6,Chella,2016-06-15 20:00:00,2016-06-17 09:00:00
7,Gatova,2017-06-28 19:00:00,2017-06-30 17:00:00
8,Llutxent,2018-08-06 14:30:00,2018-08-08 20:00:00
9,Montán,2015-07-07 20:30:00,2015-07-10 10:00:00


In [7]:
print(df_time.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Incendio  12 non-null     object
 1   Inicio    12 non-null     object
 2   Fin       12 non-null     object
dtypes: object(3)
memory usage: 416.0+ bytes
None


# 1. Obtenemos el campo Fecha Completa

In [8]:
df = pd.read_csv(path+'\\'+os.listdir(path)[0], sep=';')
df

,Fecha,Hora,Velocidad,Direccion,Temperatura,Humedad
0,2016-06-16,00:00:00,1.08,247.0,NaN,41.0
1,2016-06-16,01:00:00,0.36,234.0,NaN,46.0
2,2016-06-16,02:00:00,0.72,259.0,NaN,49.0
3,2016-06-16,03:00:00,0.36,237.0,NaN,54.0
4,2016-06-16,04:00:00,0.72,232.0,NaN,53.0
...,...,...,...,...,...,...
67,2016-06-18,19:00:00,3.24,45.0,23.4,50.0
68,2016-06-18,20:00:00,1.08,28.0,20.9,62.0
69,2016-06-18,21:00:00,1.44,36.0,19.5,72.0
70,2016-06-18,22:00:00,0.72,24.0,18.7,77.0


In [9]:
df['Fecha_Completa'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'], format='%Y/%m/%d %H:%M')

In [10]:
df

,Fecha,Hora,Velocidad,Direccion,Temperatura,Humedad,Fecha_Completa
0,2016-06-16,00:00:00,1.08,247.0,NaN,41.0,2016-06-16 00:00:00
1,2016-06-16,01:00:00,0.36,234.0,NaN,46.0,2016-06-16 01:00:00
2,2016-06-16,02:00:00,0.72,259.0,NaN,49.0,2016-06-16 02:00:00
3,2016-06-16,03:00:00,0.36,237.0,NaN,54.0,2016-06-16 03:00:00
4,2016-06-16,04:00:00,0.72,232.0,NaN,53.0,2016-06-16 04:00:00
...,...,...,...,...,...,...,...
67,2016-06-18,19:00:00,3.24,45.0,23.4,50.0,2016-06-18 19:00:00
68,2016-06-18,20:00:00,1.08,28.0,20.9,62.0,2016-06-18 20:00:00
69,2016-06-18,21:00:00,1.44,36.0,19.5,72.0,2016-06-18 21:00:00
70,2016-06-18,22:00:00,0.72,24.0,18.7,77.0,2016-06-18 22:00:00


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Fecha           72 non-null     object        
 1   Hora            72 non-null     object        
 2   Velocidad       72 non-null     float64       
 3   Direccion       72 non-null     float64       
 4   Temperatura     40 non-null     float64       
 5   Humedad         70 non-null     float64       
 6   Fecha_Completa  72 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 4.1+ KB


In [12]:
df_time.iloc[0][1]

Timestamp('2016-06-16 17:00:00')

# 2. Ubicamos los NANs

In [13]:
number_files = len(os.listdir(path))
path_save = 'C:\\Users\\joanb\\Desktop\\data_incendios\\clima\\process_data\\interpolated_data\\'

for file, i in zip(os.listdir(path), range(number_files)):

    print(file)
    df = pd.read_csv(path + '\\' + file, sep=';')
    df['Fecha_Completa'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'], format='%Y/%m/%d %H:%M')
    df = df[(df['Fecha_Completa'] >= df_time.iloc[i][1]) & (df['Fecha_Completa'] <= df_time.iloc[i][2])]
    #df['Theta'] = df['Direccion'].astype('float') * (2*np.pi/360) # Convert to radians
    
    # Calcular la cantidad de segundos transcurridos
    timestamps = df['Fecha_Completa'] - df['Fecha_Completa'].iloc[0]
    timestamps = timestamps.dt.total_seconds()

    # Calculamos los datos de los splines para cada incendio: ordenadas y abscisas.
    n = df.Fecha.unique().size
    xp = n * minmax_scale(timestamps.values) # abscisas del spline
    yp_T = df['Temperatura'].values # ordenadas del spline de Temperatura
    yp_H = df['Humedad'].values # ordenadas del spline de Humedad
    yp_V = df['Velocidad'].values # ordenadas del spline de Velocidad
    yp_D = df['Direccion'].values  # ordenadas del spline de Direccion

    # ¿Dónde hay NAN?

    if np.isnan(yp_T).any():
        print('Hay ', np.isnan(yp_T).sum(), ' NAN en Temperatura en el file: ', file)
        print('En las posiciones: ', np.where(np.isnan(yp_T) == True))
        # replace nan values with the mean of the array
        #yp_T[np.isnan(yp_T) == True] = np.nanmean(yp_T)
    if np.isnan(yp_H).any():
        print('Hay ', np.isnan(yp_H).sum(), ' NAN en Humedad en el file: ', file)
        print('En las posiciones: ', np.where(np.isnan(yp_H) == True))
        #yp_H[np.isnan(yp_H) == True] = np.nanmean(yp_H)
    if np.isnan(yp_V).any():
        print('Hay NAN en Velocidad en el file: ', file)
    if np.isnan(yp_D).any():
        print('Hay NAN en Direccion en el file: ', file)

data_Alzira
Hay  15  NAN en Temperatura en el file:  data_Alzira
En las posiciones:  (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=int64),)
Hay  2  NAN en Humedad en el file:  data_Alzira
En las posiciones:  (array([36, 37], dtype=int64),)
data_Artana
data_Beneixama
data_Beniarda
data_Bolulla
data_Carcaixent
data_Chella
data_Gatova
data_Llutxent
data_Montan
Hay  7  NAN en Humedad en el file:  data_Montan
En las posiciones:  (array([30, 31, 32, 33, 55, 56, 57], dtype=int64),)
data_ValldEbo
data_VallGallinera


# 3. Reemplazamos los Nan y extrapolamos los datos climáticos.

In [14]:
number_files = len(os.listdir(path))
path_save = 'C:\\Users\\joanb\\Desktop\\data_incendios\\clima\\process_data\\interpolated_data\\'
path_save_eps = 'C:\\Users\\joanb\\Desktop\\data_incendios\\clima\\process_data\\interp_fig\\eps\\'
path_save_png = 'C:\\Users\\joanb\\Desktop\\data_incendios\\clima\\process_data\\interp_fig\\png\\'

# Define el título de cada subplot
titles = ['Temperatura', 'Humedad', 'Velocidad', 'Dirección']

for file, i in zip(os.listdir(path), range(number_files)):

    print(file)

    df = pd.read_csv(path + '\\' + file, sep=';')
    df['Fecha_Completa'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'], format='%Y/%m/%d %H:%M')
    df = df[(df['Fecha_Completa'] >= df_time.iloc[i][1]) & (df['Fecha_Completa'] <= df_time.iloc[i][2])]
    #df['Theta'] = df['Direccion'].astype('float') * (2*np.pi/360) # Convert to radians
    
    # Calcular la cantidad de segundos transcurridos
    timestamps = df['Fecha_Completa'] - df['Fecha_Completa'].iloc[0]
    timestamps = timestamps.dt.total_seconds()

    # Calculamos los datos de los splines para cada incendio: ordenadas y abscisas.
    n = df.Fecha.unique().size
    xp = n * minmax_scale(timestamps.values) # abscisas del spline
    yp_T = df['Temperatura'].values # ordenadas del spline de Temperatura
    yp_H = df['Humedad'].values # ordenadas del spline de Humedad
    yp_V = df['Velocidad'].values # ordenadas del spline de Velocidad
    yp_D = df['Direccion'].values  # ordenadas del spline de Direccion

    # ¿Dónde hay NAN?

    if np.isnan(yp_T).any():
        print('Hay NAN en Temperatura en el file: ', file)
        yp_T[np.isnan(yp_T) == True] = np.nanmean(yp_T)
    if np.isnan(yp_H).any():
        print('Hay NAN en Humedad en el file: ', file)
        yp_H[np.isnan(yp_H) == True] = np.nanmean(yp_H)
    if np.isnan(yp_V).any():
        print('Hay NAN en Velocidad en el file: ', file)
    if np.isnan(yp_D).any():
        print('Hay NAN en Direccion en el file: ', file)

    # Creamos los splines
    p_T = CubicSpline(xp, yp_T)
    p_H = CubicSpline(xp, yp_H)
    p_V = CubicSpline(xp, yp_V)
    p_D = CubicSpline(xp, yp_D)

    # Creamos los nuevos datos

    # Queremos los datos unificados para cada media hora. Es decir para la 01:00, 01:30, 02:00, etc.
    # Para ello, calculamos la cantidad de datos que hay en cada media hora.
    n_x = 48 * n + 1 # 48 datos por día + 1 dato para el último día (el de cierre)
    x_data = np.linspace(0, n, n_x) # abscisas de los nuevos datos

    # Calculamos los nuevos datos
    y_data_T = p_T(x_data)
    y_data_H = p_H(x_data)
    y_data_V = p_V(x_data)
    y_data_D = p_D(x_data)

################################################################################################################

    # Hacemos figuras para ver los resultados
    # Crea los cuatro subplots
    fig, axs = plt.subplots(2, 2, figsize=(8, 10))
    x_fig = np.linspace(0, n, 1000) # abscisas para graficar la curva spline

    # 1. Temperatura

    # Grafica los datos en azul
    axs[0, 0].scatter(xp, yp_T, s=8, c='b')
    # Grafica la curva spline en rojo
    axs[0, 0].plot(x_fig, p_T(x_fig), 'r--')
    # Agrega el título al subplot
    axs[0,0].set_title(titles[0])
    # 2. Humedad

    axs[0, 1].scatter(xp, yp_H, s=8, c='b')
    axs[0, 1].plot(x_fig, p_H(x_fig), 'r--')
    axs[0, 1].set_title(titles[1])
    # 3. Velocidad

    axs[1, 0].scatter(xp, yp_V, s=8, c='b')
    axs[1, 0].plot(x_fig, p_V(x_fig), 'r--')
    axs[1, 0].set_title(titles[2])

    # 4. Dirección

    axs[1, 1].scatter(xp, yp_D, s=8, c='b')
    axs[1, 1].plot(x_fig, p_D(x_fig), 'r--')
    axs[1, 1].set_title(titles[3])


    # Ajusta el espacio entre los subplots
    plt.subplots_adjust(hspace=0.5)
    plt.savefig(path_save_eps + 'fig_' + file + '.eps', format='eps')
    plt.savefig(path_save_png + 'fig_' + file + '.png', format='png')
    plt.close()
    # Muestra la gráfica
    #plt.show()

#####################################################################################################################

    # Creamos el nuevo dataframe
    df_new = pd.DataFrame(index=range(n_x), columns=['Fecha', 'X', 'Temperatura', 'Humedad', 'Velocidad', 'Direccion'])
    df_new['Fecha'] = pd.date_range(start=df_time.iloc[i][1], periods=n_x, freq='30min')
    df_new['X'] = x_data
    df_new['Temperatura'] = y_data_T
    df_new['Humedad'] = y_data_H
    df_new['Velocidad'] = y_data_V
    df_new['Direccion'] = y_data_D

    # Guardamos el nuevo dataframe
    df_new.to_csv(path_save + file, sep = ";", decimal = ".", index = False) 

data_Alzira
Hay NAN en Temperatura en el file:  data_Alzira
Hay NAN en Humedad en el file:  data_Alzira
data_Artana
data_Beneixama
data_Beniarda
data_Bolulla
data_Carcaixent
data_Chella
data_Gatova
data_Llutxent
data_Montan
Hay NAN en Humedad en el file:  data_Montan
data_ValldEbo
data_VallGallinera
